In [1]:
from pathlib import Path
import sys
import cobra
import os
import pandas as pd
import re

In [2]:
path_root = Path("C:/Users/tobyl/OneDrive - The University of Manchester/Bioinformatics Masters/Research Project 2/Pycomo/PyCoMo/src")
sys.path.append(str(path_root))
import pycomo


### Creating community model
1. Load the member models

In [3]:
# loading member models
initial_path = Path().absolute().parent

model_member_dir = initial_path / r"old_files/models_members_test/Toy"
named_models = pycomo.load_named_models_from_dir(model_member_dir)

In [4]:
# checking modesl that have been loaded
print(named_models)

{'Faecalibacterium': <Model Faecalibacterium_prausnitzii_M21_2 at 0x7f8755c3f050>, 'Bifidobacterium': <Model Bifidobacterium_adolescentis_ATCC_15703 at 0x7f8755968440>, 'Bacteroides': <Model Bacteroides_uniformis_ATCC_8492 at 0x7f8755638bc0>}


In [ ]:
# checking if I can edit the models
model1 = named_models['Escherichia_coli_UTI89_UPEC']
for reaction in model1.exchanges:
    print(reaction.name)
    print(reaction)
    print(reaction.bounds)

2. checking the objective function of the loaded models

In [5]:
# if all have a biomass objective function then it is working
for model in named_models.values():
    print(model.objective)

Maximize
1.0*biomass224 - 1.0*biomass224_reverse_c48a3
Maximize
1.0*biomass345 - 1.0*biomass345_reverse_e128f
Maximize
1.0*biomass536 - 1.0*biomass536_reverse_50d4e


In [7]:
# creating the single organism models in PyCoMo format
single_org_models = []
for name, model in named_models.items():
    print(name)
    single_org_model = pycomo.SingleOrganismModel(model, name)
    single_org_models.append(single_org_model)

Faecalibacterium
Bifidobacterium
Bacteroides


3. Creating the community model

In [8]:
community_name = "ten_gutControl_model"
com_model_obj = pycomo.CommunityModel(single_org_models, community_name)

In [11]:
##### medium creation code
import re

parent_path = Path().absolute().parent
diet_filepath = parent_path / r"diet_info/western_fluxes.tsv"
diet_df = pd.read_csv(diet_filepath, sep = "\t")
diet_df.loc[:, 'Reaction'] = diet_df['Reaction'].apply(lambda reaction: re.split(r"\[|\(", reaction)[0])
diet_reactionID_list = list(diet_df.loc[:, 'Reaction'])


#creating the medium with the diet values
medium_exchange_dict = {}
for medium_reaction in com_model_obj.model.exchanges:
    medium_reaction_id = medium_reaction.id

    # ensuring medium_reaction_id matches ids from diet_reaction_ID_list
    medium_reaction_parts = medium_reaction_id.split("_")
    if medium_reaction_parts[1] == "":
        del(medium_reaction_parts)[1]
    del(medium_reaction_parts)[-1]
    reaction_id = "_".join(medium_reaction_parts)

    if reaction_id in medium_exchange_dict.values():
        continue
    elif reaction_id in diet_reactionID_list:
        flux = diet_df.loc[diet_df['Reaction'] == reaction_id, 'Flux Value'].values
        medium_exchange_dict[medium_reaction_id] = float(flux[0])
    else:
            medium_exchange_dict[medium_reaction_id] = 1e-6
print(medium_exchange_dict)

        
           
           
       

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
{'EX__12dgr180_medium': 1e-06, 'EX__3mop_medium': 1e-06, 'EX_Lcyst_medium': 1e-06, 'EX_Lcystin_medium': 1e-06, 'EX_MGlcn100_medium': 1e-06, 'EX_MGlcn100_rl_medium': 1e-06, 'EX_MGlcn101_medium': 1e-06, 'EX_MGlcn101_rl_medium': 1e-06, 'EX_MGlcn102_medium': 1e-06, 'EX_MGlcn102_rl_medium': 1e-06, 'EX_MGlcn104_medium': 1e-06, 'EX_MGlcn104_rl_medium': 1e-06, 'EX_MGlcn105_medium': 1e-06, 'EX_MGlcn105_rl_medium': 1e-06, 'EX_MGlcn106_medium': 1e-06, 'EX_MGlcn106_rl_medium': 1e-06, 'EX_MGlcn107_medium': 1e-06, 'EX_MGlcn107_rl_medium': 1e-06, 'EX_MGlcn109_medium': 1e-06, 'EX_MGlcn109_rl_medium': 1e-06, 'EX_MGlcn110_medium': 1e-06, 'EX_MGlcn110_rl_medium': 1e-06, 'EX_MGlcn111_medium': 1e-06, 'EX_MGlcn111_rl_medium': 1e-06, 'EX_MGlcn112_medium': 1e-06, 'EX_MGlcn112_rl_medi

In [ ]:
# calling the community model to make it
com_model_obj.model

In [ ]:
##### medium creation code


model_members = com_model_obj.member_models
single_org_model = model_members[0]
model = single_org_model.model
for reaction in model.exchanges:
    print(reaction.id)
    print(reaction)
    print(reaction.bounds)

### Getting the Summary and Report for the model

In [ ]:
com_model_obj.summary()

- In report, need to check any unbalanced reactions that are not sink/demand (SINK, DM), Exchange (EX), Transport (T_ or TR_) or BIOMASS

In [ ]:
com_model_obj.report()

- Checking if there are any thermodynamically infeasible cycles

In [ ]:
# com_model_obj.get_loops()
# taking too long to run so I stopped it
# could be a problem and will have to 

## Step 3: Converting to fixed abundances as we are modelling specific conditions

In [ ]:
# creating the abundance_dict
abundance_dict = com_model_obj.generate_equal_abundance_dict()
print(sum(abundance_dict.values()))
print(abundance_dict)

In [ ]:
# converting to fixed abundance model and then adding the abundances from abundance_dict
com_model_obj.convert_to_fixed_abundance()
com_model_obj.apply_fixed_abundance(abundance_dict)

# summarising the model
com_model_obj.summary()





In [ ]:
summary = com_model_obj.summary()

up_flux = summary.uptake_flux # how to get the uptake flux
sec_flux = summary.secretion_flux # how to get the secretion flux

# can get the objective value and status from this!
solution = com_model_obj.model.optimize()
print(solution.objective_value) # how to get the objective_value
print(solution.status) # How to get the status

print(up_flux.head())
print(sec_flux.head())


- Now, we can check all the potential exchange metabolic fluxes and cross-feeding interactions

In [ ]:
MES = com_model_obj.potential_metabolite_exchanges()

In [ ]:
MES.to_csv("mes_results.csv")